In [1]:
from common import *

cursor = connect()

# INSTEAD OF TRIGGERS

`instead of` 触发器是一种特殊类型的触发器，它拦截视图上的插入、更新和删除操作。这意味着当您对视图执行 INSERT、UPDATE 或 DELETE 语句时，PostgreSQL 不会直接执行该语句。相反，它执行 INSTEAD OF 触发器中定义的语句。

语法
```
CREATE TRIGGER trigger_name
INSTEAD OF INSERT OR UPDATE OR DELETE 
ON table_name
FOR EACH ROW 
EXECUTE FUNCTION fn_trigger;
```

In [2]:
sql = """
CREATE TABLE employees (
    employee_id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL
);

CREATE TABLE salaries (
    employee_id INT,
    effective_date DATE NOT NULL,
    salary DECIMAL(10, 2) NOT NULL DEFAULT 0, 
    PRIMARY KEY (employee_id, effective_date),
    FOREIGN KEY (employee_id) REFERENCES employees(employee_id)
);

INSERT INTO employees (name) 
VALUES 
   ('Alice'), 
   ('Bob')
RETURNING *;

INSERT INTO salaries 
VALUES 
   (1, '2024-03-01', 60000.00), 
   (2, '2024-03-01', 70000.00)
RETURNING *;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x2d3fb519550>

In [3]:
sql = """
CREATE VIEW employee_salaries 
AS
SELECT e.employee_id, e.name, s.salary, s.effective_date
FROM employees e
JOIN salaries s ON e.employee_id = s.employee_id;

CREATE OR REPLACE FUNCTION update_employee_salaries()
RETURNS TRIGGER AS 
$$
DECLARE
    p_employee_id INT;
BEGIN
    IF TG_OP = 'INSERT' THEN
	    -- insert a new employee 
        INSERT INTO employees(name) 
        VALUES (NEW.name)
	    RETURNING employee_id INTO p_employee_id;

	    -- insert salary for the employee
        INSERT INTO salaries(employee_id, effective_date, salary) 
        VALUES (p_employee_id, NEW.effective_date, NEW.salary);
    ELSIF TG_OP = 'UPDATE' THEN
        UPDATE salaries 
        SET salary = NEW.salary 
        WHERE employee_id = NEW.employee_id;

    ELSIF TG_OP = 'DELETE' THEN
        DELETE FROM salaries 
	    WHERE employee_id = OLD.employee_id;
        DELETE FROM employees 
	    WHERE employee_id = OLD.employee_id;
    END IF;
    RETURN NULL;
END;
$$ 
LANGUAGE plpgsql;

CREATE TRIGGER instead_of_employee_salaries
INSTEAD OF INSERT OR UPDATE OR DELETE 
ON employee_salaries
FOR EACH ROW 
EXECUTE FUNCTION update_employee_salaries();
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x2d3fb519550>

## 往视图插入数据

In [4]:
sql = """
INSERT INTO employee_salaries (name, salary, effective_date)
VALUES ('Charlie', 75000.00, '2024-03-01');
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x2d3fb519550>

In [5]:
sql = """
SELECT * FROM employees;
"""
run_sql(cursor, sql)

   employee_id     name
0            1    Alice
1            2      Bob
2            3  Charlie


In [6]:
sql = """
SELECT * FROM salaries;
"""
run_sql(cursor, sql)

   employee_id effective_date    salary
0            1     2024-03-01  60000.00
1            2     2024-03-01  70000.00
2            3     2024-03-01  75000.00


## 往视图更新数据

In [7]:
sql = """
UPDATE employee_salaries
SET salary = 95000
WHERE employee_id = 3;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x2d3fb519550>

In [8]:
sql = """
SELECT * FROM salaries;
"""
run_sql(cursor, sql)

   employee_id effective_date    salary
0            1     2024-03-01  60000.00
1            2     2024-03-01  70000.00
2            3     2024-03-01  95000.00


## 从视图删除数据

In [9]:
sql = """
DELETE FROM employee_salaries
WHERE employee_id = 3;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x2d3fb519550>

In [10]:
sql = """
SELECT * FROM employees;
"""
run_sql(cursor, sql)

   employee_id   name
0            1  Alice
1            2    Bob


In [11]:
sql = """
SELECT * FROM salaries;
"""
run_sql(cursor, sql)

   employee_id effective_date    salary
0            1     2024-03-01  60000.00
1            2     2024-03-01  70000.00
